In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import pickle as pk
import numpy as np

CIFAR_DIR = '../../datasets/cifar-10-batches-py'
dirs = os.listdir( CIFAR_DIR )
print(dirs)

Instructions for updating:
non-resource variables are not supported in the long term
['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [ ]:
def load_data(filename):
    """read data from data file"""
    with open(filename,'rb') as f:
        data = pk.load(f,encoding='iso-8859-1')
        return (data['data'],data['labels'])

class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data ,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
          
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1  #归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        """打乱数据集"""
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._lable = self._labels[p]
        
    def next_batch(self,batch_size):
        """ 随机取batch_size个训练样本  """
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = self._indicator + batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch_size is larger than examples")
            
        batch_data = self._data[self._indicator:end_indicator]
        batch_lables = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data , batch_lables


train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' %i) for i in range (1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
tets_data = CifarData(test_filenames,False)


In [ ]:
def Inception_block(x,
                   output_channel_for_each_path,
                   name):
    '''
    :param x: tensor
    :param output_channel_for_each_path: 每一个组输出通道数 结构: eg: [10, 20, 5]
    :param name: 防止重命名
    :return: 已经合并好的tensor
    '''

    conv1_1 = tf.layers.conv2d(x,
                                output_channel_for_each_path[0],
                                (1, 1),
                                strides = (1, 1),
                                padding = 'same',
                                activation = tf.nn.relu,
                                name = 'conv1_1'
                                   )
        conv3_3 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[1],
                                   (3, 3),
                                   strides=(1, 1),
                                   padding='same',
                                   activation=tf.nn.relu,
                                   name='conv3_3'
                                   )
        conv5_5 = tf.layers.conv2d(x,
                                   output_channel_for_each_path[2],
                                   (5, 5),
                                   strides=(1, 1),
                                   padding='same',
                                   activation=tf.nn.relu,
                                   name='conv5_5'
                                   )
        max_pooling = tf.layers.max_pooling(conv5_5,
                                           (2,2),
                                           (2,2),
                                           name = "max-pooling")
        
        max_pooling_shape = max_pooling.get_shape().as_list[1:]
        input_shape = x.get_shape().as_list[1:]
        padding_width = (input_shape[0]-max_pooling_shape[0]) //2
        padding_height = (input_shape[1]-max_pooling_shape[1]) //2
            
        padded_pooling = tf.pad(max_pooling,
                                [[0,0],
                                 [padding_width,padding_width],
                                 [padding_height,padding_height],
                                [0,0]])
        
    # 接下来需要将 三个 卷积层 和 一个 池化层 拼接在一起,axis 是指定按照第几维合并
    # 本实验其实就是把第四维进行合并
    
    concat_layer = tf.concat([conv1_1],[con])
    
    
    
    
tf.reset_default_graph()  
x = tf.placeholder(tf.float32,[None,3072])  #定义一个空的数组
y = tf.placeholder(tf.int64,[None])

x_image = tf.reshape(x,[-1,3,32,32])
# 将最开始的向量式的图片,转为真实的图片类型
x_image = tf.transpose(x_image, perm=[0,2,3,1])

y_ = res_net(x_image,[2,3,2],32,10)

loss = tf.losses.sparse_softmax_cross_entropy(labels =y,logits = y_)
# # 使用交叉熵 设置损失函数,该api,做了三件事儿 1. y_ -> softmax 2. y -> one_hot 3. loss = ylogy
predict  = tf.argmax(y_ ,1)
correct_predict = tf.equal(predict,y)
accuracy = tf.reduce_mean(tf.cast(correct_predict,tf.float32))

with tf.name_scope('train_op'): 
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss) 

In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data , batch_labels = train_data.next_batch(batch_size)
        loss_val,acc_val,_=sess.run(
            [loss,accuracy,train_op],
            feed_dict = {x:batch_data,
                         y:batch_labels})
        if (i+1) % 500  == 0:
            print('[Train] Step: %d,Loss:%4.5f,acc:%4.5f'
                  %(i+1,loss_val,acc_val))
        
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames,False)
            all_test_acc = []
            for j in range(test_steps):
                test_batch_data , test_batch_labels  = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict={x:test_batch_data, y:test_batch_labels})
                all_test_acc.append(test_acc_val)
            
            test_acc = np.mean(all_test_acc)
            #print("[Test] Step: %d , Acc: %4.5f" % (i+1,test_acc))